# 结合Claude使用计算器工具

在本教程中，我们将演示如何为Claude提供一个简单的计算器工具，让它能够根据用户输入执行算术运算。我们将定义计算器工具，并展示Claude如何与之交互来解决数学问题。

## 步骤1：设置环境

首先，让我们安装所需的库并设置Claude API客户端。

In [ ]:
%pip install anthropic

In [11]:
from anthropic import Anthropic

client = Anthropic()
MODEL_NAME = "claude-opus-4-1"

## 步骤2：定义计算器工具

我们将定义一个能够执行基本算术运算的简单计算器工具。该工具将接受数学表达式作为输入并返回结果。

注意，我们对输出的表达式调用了```eval```。这是一种不良实践，通常不应该使用，但出于演示目的我们这样做。

In [2]:
import re


def calculate(expression):
    # Remove any non-digit or non-operator characters from the expression
    expression = re.sub(r"[^0-9+\-*/().]", "", expression)

    try:
        # Evaluate the expression using the built-in eval() function
        result = eval(expression)
        return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
        return "Error: Invalid expression"


tools = [
    {
        "name": "calculator",
        "description": "A simple calculator that performs basic arithmetic operations.",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate (e.g., '2 + 3 * 4').",
                }
            },
            "required": ["expression"],
        },
    }
]

在这个例子中，我们定义了一个calculate函数，它接受数学表达式作为输入，使用正则表达式删除任何非数字或非运算符字符，然后使用内置的eval()函数计算表达式。如果计算成功，结果将作为字符串返回。如果在计算过程中发生错误，则返回错误消息。

然后，我们定义一个计算器工具，它有一个输入模式，期望一个字符串类型的expression属性。

## 步骤3：与Claude交互

现在，让我们看看Claude如何与计算器工具交互来解决数学问题。

In [36]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "calculator":
        return calculate(tool_input["expression"])


def chat_with_claude(user_message):
    print(f"\n{'=' * 50}\nUser Message: {user_message}\n{'=' * 50}")

    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        messages=[{"role": "user", "content": user_message}],
        tools=tools,
    )

    print("\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")

    if message.stop_reason == "tool_use":
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"Tool Result: {tool_result}")

        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": tool_result,
                        }
                    ],
                },
            ],
            tools=tools,
        )
    else:
        response = message

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )
    print(response.content)
    print(f"\nFinal Response: {final_response}")

    return final_response

## 步骤4：试试看！

现在让我们尝试给Claude几个数学问题，因为它已经可以使用计算器了。

In [37]:
chat_with_claude("What is the result of 1,984,135 * 9,343,116?")
chat_with_claude("Calculate (12851 - 593) * 301 + 76")
chat_with_claude("What is 15910385 divided by 193053?")


User Message: What is the result of 1,984,135 * 9,343,116?

Initial Response:
Stop Reason: tool_use
Content: [ContentBlock(text='<thinking>\nThe calculator function is the relevant tool to answer this request, since it involves evaluating a mathematical expression.\n\nThe required parameter for the calculator function is:\nexpression: The mathematical expression to evaluate.\n\nThe human has directly provided the full expression to evaluate in their request: "1,984,135 * 9,343,116". This contains all the information needed for the required expression parameter.\n\nSo I have the necessary information to invoke the calculator tool.\n</thinking>', type='text'), ContentBlockToolUse(id='toolu_01V2mzqp5qkB5QucRFjJUJLD', input={'expression': '1984135 * 9343116'}, name='calculator', type='tool_use')]

Tool Used: calculator
Tool Input: {'expression': '1984135 * 9343116'}
Tool Result: 18538003464660
[ContentBlock(text='Therefore, the result of 1,984,135 * 9,343,116 is 18,538,003,464,660.', type

'So 15910385 divided by 193053 equals 82.41459599177428.'